In [1]:
%run model.ipynb

Knocking out reactions mentioned in the paper by Wu et al (Wu et al., 2019):
- LDH which converts pyruvate to lactate
- PTA which converts Acetyl-CoA to Acetyl phosphate

### Finding knockout targets

#### Finding the LDH reaction:

In [2]:
model.reactions.query("LDH")

[<Reaction BETALDHx at 0x1f67cc75e50>,
 <Reaction LDH2 at 0x1f67cf77040>,
 <Reaction LDH_L at 0x1f67cfa2df0>,
 <Reaction VALDHr at 0x1f67d222850>]

Checking if LDH_L is the one producing lactate

In [3]:
model.reactions.LDH_L.metabolites

{<Metabolite lac__L_c at 0x1f67c702eb0>: -1.0,
 <Metabolite nad_c at 0x1f67c717f70>: -1.0,
 <Metabolite h_c at 0x1f67c6d0310>: 1.0,
 <Metabolite nadh_c at 0x1f67c72f310>: 1.0,
 <Metabolite pyr_c at 0x1f67c77c6a0>: 1.0}

In [4]:
model.metabolites.lac__L_c.name

'L-Lactate'

Yes. Seems like LDH_L is the reaction we're looking for.

#### Finding the PTA reaction

In [5]:
model.reactions.query("PTA")

[<Reaction APTA1i at 0x1f67cb87790>,
 <Reaction ASPTA at 0x1f67cc6a340>,
 <Reaction GF6PTA at 0x1f67ce0db20>,
 <Reaction PTAr at 0x1f67d055d30>,
 <Reaction PTA2 at 0x1f67d055370>]

Checking if PTAr is the reaction Acetyl phosphate:

In [6]:
model.reactions.PTAr.metabolites

{<Metabolite accoa_c at 0x1f67c5b0bb0>: -1.0,
 <Metabolite pi_c at 0x1f67c702280>: -1.0,
 <Metabolite actp_c at 0x1f67c5e8190>: 1.0,
 <Metabolite coa_c at 0x1f67c5e8dc0>: 1.0}

In [7]:
model.metabolites.actp_c.name

'Acetyl phosphate'

Yes. Seems like PTAr is the reaction we're looking for.

### Knocking out reactions

Now we'll try knocking it out each of the reactions and see what happens to the growth and surfactin yield:

Knockout of LDH_L:

In [21]:
with model:
    model.reactions.LDH_L.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.LDH_L.lower_bound, model.reactions.LDH_L.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    model.objective = model.reactions.DM_surfactin_c
    surfactin_production = model.optimize().objective_value
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_glc__D_e.flux))

LDH_L's bounds inside the with statement:
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.07461282448997734
Theoretical max. yield [mmol-succ / mmol-ac]: 0.04388989675881021


Knockout of PTAr:

In [22]:
with model:
    model.reactions.PTAr.bounds = 0, 0
    print("LDH_L's bounds inside the with statement:")
    print(model.reactions.PTAr.lower_bound, model.reactions.PTAr.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    model.objective = model.reactions.DM_surfactin_c
    surfactin_production = model.optimize().objective_value
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", surfactin_production / (-1*model.reactions.EX_glc__D_e.flux))

LDH_L's bounds inside the with statement:
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.0743950621585464
Theoretical max. yield [mmol-succ / mmol-ac]: 0.043761801269733176


In both cases we see that the yield does not increase significantly